In [ ]:
import os
os.environ['NOTEBOOK_MODE'] = '1'

import pandas as pd

from collections import defaultdict
from utils import *
from inspection_utils import *

from dataset_paths import IMAGENET_PATH, SALIENT_IMAGENET_PATH

In [ ]:
noise_stds_list = [0.25]
model_name_list = ['resnet18', 'resnet34', 'resnet50', 'vgg19_bn', 'inception_v3', 'mnasnet_100', 
                   'wide_resnet50_2', 'efficientnet_b0', 'efficientnet_b4', 'tf_efficientnet_b7',
                   'clip_vit_b16', 'clip_vit_b32']

In [ ]:
csv_path = 'mturk_results/discover_spurious_features.csv'
mturk_results_discover = MTurk_Results(csv_path)

class_to_features_dict = mturk_results_discover.class_to_features_dict
core_spurious_dict = mturk_results_discover.core_spurious_dict
feature_rank_dict = mturk_results_discover.feature_rank_dict

In [ ]:
batch_size = 32
num_decimal = 3

failure_analysis_instance = NoiseSensitivityAnalysis(IMAGENET_PATH,
                                                     SALIENT_IMAGENET_PATH, 
                                                     batch_size=batch_size)

class_df = pd.read_csv('class_metadata.csv')
class_indices = class_df['class_index'].tolist()
synsets = class_df['synset'].tolist()
class_names = dict(zip(class_indices, synsets))

inspection_results_root = 'evaluation_results'
os.makedirs(inspection_results_root, exist_ok=True)
inspection_results_path = os.path.join(inspection_results_root, 'noise_in_single_feature')
os.makedirs(inspection_results_path, exist_ok=True)

for model_name in model_name_list:
    inspection_model, _ = load_inspection_model(model_name)
    
    print_with_stars(" Model name: {:s} ".format(model_name))
    model_results_path = os.path.join(inspection_results_path, model_name)
    os.makedirs(model_results_path, exist_ok=True)

    
    for noise_std in noise_stds_list:
        inspection_results_dict = defaultdict(list)
        for class_index in range(1000):
            full_class_name = class_names[class_index]

            feature_indices = class_to_features_dict[class_index]
            for feature_index in feature_indices:
                key = str(class_index) + '_' + str(feature_index)
                feature_label = core_spurious_dict[key]
                feature_rank = feature_rank_dict[key]

                acc_tuple = failure_analysis_instance.compute_noisy_acc(inspection_model, 
                                                                        class_index, [feature_index], 
                                                                        noise_std=noise_std)
                clean_acc, noisy_acc = acc_tuple
                
                inspection_results_dict['class_index'].append(class_index)
                inspection_results_dict['feature_index'].append(feature_index)
                inspection_results_dict['feature_rank'].append(feature_rank)
                inspection_results_dict['feature_label'].append(feature_label)
                inspection_results_dict['class_name'].append(full_class_name)

                inspection_results_dict['clean_accuracy'].append(
                    round(100 * clean_acc, num_decimal))
                inspection_results_dict['noisy_accuracy'].append(
                    round(100 * noisy_acc, num_decimal))
                
            inspection_results_df = pd.DataFrame.from_dict(inspection_results_dict)
            curr_results_path = os.path.join(model_results_path, 'std_' + str(noise_std) + '.csv')
            inspection_results_df.to_csv(curr_results_path, index=False)